# Proposal Document

In [48]:
#Read packages into R
library(tidyverse)
library(repr)
library(tidymodels)
library(GGally)
options(repr.matrix.max.rows = 6)

Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2



In [46]:
pulsar_base_data = read_csv("pulsar_data_train.csv", skip = 1,
                            col_names = c("mean_prof", "sd_prof", 
                                          "kurt_prof", "skew_prof", "mean_DMSNR",
                                          "sd_DMSNR","kurt_DMSNR","skew_DMSNR", "target_class")) %>% 
                    mutate(target_class = as_factor(target_class)) %>% 
                    mutate(target_class = fct_recode(target_class,
                                                    "non_pulsar" = '0',
                                                    "pulsar" = '1')) %>% 
                    drop_na()
           
colnames(pulsar_base_data)
head(pulsar_base_data)

Parsed with column specification:
cols(
  mean_prof = col_double(),
  sd_prof = col_double(),
  kurt_prof = col_double(),
  skew_prof = col_double(),
  mean_DMSNR = col_double(),
  sd_DMSNR = col_double(),
  kurt_DMSNR = col_double(),
  skew_DMSNR = col_double(),
  target_class = col_double()
)



[1] "mean_prof"    "sd_prof"      "kurt_prof"    "skew_prof"    "mean_DMSNR"  
[6] "sd_DMSNR"     "kurt_DMSNR"   "skew_DMSNR"   "target_class"

mean_prof,sd_prof,kurt_prof,skew_prof,mean_DMSNR,sd_DMSNR,kurt_DMSNR,skew_DMSNR,target_class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
121.15625,48.37297,0.3754847,-0.01316549,3.168896,18.39937,7.449874,65.15930,non_pulsar
76.96875,36.17556,0.7128979,3.38871856,2.399666,17.57100,9.414652,102.72297,non_pulsar
130.58594,53.22953,0.1334083,-0.29724164,2.743311,22.36255,8.508364,74.03132,non_pulsar
84.80469,36.11766,0.8250128,3.27412537,2.790134,20.61801,8.405008,76.29113,non_pulsar
109.40625,55.91252,0.5651059,0.05624666,2.797659,19.49653,9.443282,97.37458,non_pulsar
95.00781,40.21981,0.3475781,1.15316438,2.770067,18.21774,7.851205,70.80194,non_pulsar


In [47]:
#Standardization of all predictor variables
pulsar_recipe = recipe(target_class~., data = pulsar_base_data) %>% 
                    step_scale(all_predictors()) %>% 
                    step_center(all_predictors()) %>% 
                    prep()
scaled_pulsar = bake(pulsar_recipe, pulsar_base_data)
head(scaled_pulsar)

mean_prof,sd_prof,kurt_prof,skew_prof,mean_DMSNR,sd_DMSNR,kurt_DMSNR,skew_DMSNR,target_class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
0.39018694,0.2741234,-0.09714898,-0.2869353,-0.3213805,-0.4057024,-0.19298366,-0.37556899,non_pulsar
-1.33008102,-1.5245552,0.21785037,0.2542001,-0.3472224,-0.4480902,0.23884387,-0.02829310,non_pulsar
0.75729504,0.9902911,-0.32314459,-0.3321231,-0.3356778,-0.2029055,0.03965582,-0.29354718,non_pulsar
-1.02501936,-1.5330931,0.32251767,0.2359718,-0.3341048,-0.2921741,0.01693992,-0.27265531,non_pulsar
-0.06725348,1.3859349,0.07987605,-0.2758940,-0.3338520,-0.3495605,0.24513627,-0.07773898,non_pulsar
-0.62780048,-0.9281744,-0.12320175,-0.1014080,-0.3347790,-0.4149962,-0.10477741,-0.32340283,non_pulsar


## To-do

Visualize all variables:
- boxplots
- heatmap ggpairs()
- scatterplot with colours points (a few different pairs)


Questions for TA
- Drop missing?
- mutate not working?
- How many visuals?